In [48]:
import pandas as pd
import numpy as np
import datetime

In [49]:
tripdata_months = {'202005','202006','202007'}

In [50]:
for month in tripdata_months:
    # read in csv file
    csv_data = f"data/{month}-citibike-tripdata.csv"
    df = pd.read_csv(csv_data)
    
    # split start time and start day into separate columns
    split1 = df["starttime"].str.rsplit(" ", n = 1, expand = True)
    df["startday"] = split1[0]
    df["starttime"] = split1[1]

    split2 = df["starttime"].str.rsplit(".", n = 1, expand = True)
    df["starttime"] = split2[0]
    
    # remove unknown gender
    df = df[df.gender!=0]
    
    # output to csv file
    df.to_csv(f"data/cleaned_{month}-citibike-tripdata.csv")

In [51]:
# create an empty dataframe object
final_df = pd.DataFrame()

for month in tripdata_months:
        # read in cleaned csv file
        csv_data = f"data/cleaned_{month}-citibike-tripdata.csv"
        df = pd.read_csv(csv_data)

        # concatenate/append datasets
        final_df = pd.concat([final_df,df], ignore_index=True)

In [52]:
# get the current year
d = datetime.datetime.today()
current_year = d.strftime('%Y')

In [53]:
# add age column
final_df["age"] = int(current_year) - final_df["birth year"]
final_df.head(5)

,Unnamed: 0,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,startday,age
0,0,336,00:01:06,2020-05-01 00:06:42.9560,2017,E 43 St & 2 Ave,40.750224,-73.971214,465,Broadway & W 41 St,40.755136,-73.986580,18015,Subscriber,1993,1,2020-05-01,27
1,1,853,00:01:08,2020-05-01 00:15:21.2150,3376,E 65 St & 2 Ave,40.764719,-73.962221,478,11 Ave & W 41 St,40.760301,-73.998842,35905,Subscriber,1989,1,2020-05-01,31
2,2,78,00:01:08,2020-05-01 00:02:27.0490,396,Lefferts Pl & Franklin Ave,40.680342,-73.955769,3789,Fulton St & Irving Pl,40.681860,-73.959432,33350,Subscriber,1938,2,2020-05-01,82
3,3,1185,00:02:22,2020-05-01 00:22:08.1220,3630,Frederick Douglass Blvd & W 115 St,40.803865,-73.955931,3506,Lexington Ave & E 120 St,40.801307,-73.939817,43790,Subscriber,1996,1,2020-05-01,24
4,4,272,00:02:32,2020-05-01 00:07:05.0880,3776,Central Ave & Starr Street,40.700003,-73.928340,3068,Humboldt St & Varet St,40.703172,-73.940636,39912,Subscriber,1993,1,2020-05-01,27


In [54]:
# remove Unnamed: 0 column
final_df.rename({"Unnamed: 0":"temp"}, axis="columns", inplace=True)    
final_df.drop(["temp"], axis=1, inplace=True)

In [55]:
# drop null values
final_df = final_df.dropna()

In [56]:
# reassign gender numerical value to male or female
final_df = final_df.replace({'gender': 1}, {'gender': 'Male'}, regex=True)
final_df = final_df.replace({'gender': 2}, {'gender': 'Female'},regex=True)

In [57]:
# preview before output to csv file
final_df.head(5)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,startday,age
0,336,00:01:06,2020-05-01 00:06:42.9560,2017,E 43 St & 2 Ave,40.750224,-73.971214,465,Broadway & W 41 St,40.755136,-73.986580,18015,Subscriber,1993,Male,2020-05-01,27
1,853,00:01:08,2020-05-01 00:15:21.2150,3376,E 65 St & 2 Ave,40.764719,-73.962221,478,11 Ave & W 41 St,40.760301,-73.998842,35905,Subscriber,1989,Male,2020-05-01,31
2,78,00:01:08,2020-05-01 00:02:27.0490,396,Lefferts Pl & Franklin Ave,40.680342,-73.955769,3789,Fulton St & Irving Pl,40.681860,-73.959432,33350,Subscriber,1938,Female,2020-05-01,82
3,1185,00:02:22,2020-05-01 00:22:08.1220,3630,Frederick Douglass Blvd & W 115 St,40.803865,-73.955931,3506,Lexington Ave & E 120 St,40.801307,-73.939817,43790,Subscriber,1996,Male,2020-05-01,24
4,272,00:02:32,2020-05-01 00:07:05.0880,3776,Central Ave & Starr Street,40.700003,-73.928340,3068,Humboldt St & Varet St,40.703172,-73.940636,39912,Subscriber,1993,Male,2020-05-01,27


In [62]:
# found a phenomenon of bad age or birth date input incorrectly
bad_age_df = final_df.loc[final_df["age"]>100]
bad_age_df.head(5)
bad_age_df.to_csv("data/bad_citibike-tripdata.csv", encoding='utf-8', index=False)

# drop birth year before and up to 1919
final_df = final_df.loc[final_df["birth year"]>1919]

In [63]:
# output final dataframe to csv
final_df.to_csv("data/citibike-tripdata.csv", encoding='utf-8', index=False)

In [64]:
# check no ages over 100
final_df.loc[final_df["age"]>100]

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,startday,age
